## **Bienvenido a la segunda práctica de Jupyter Notebook**

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jpcano1/MINE-4101-tareas/blob/master/lab_2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

## **Preguntas**
1. Descargue las bases de datos B.Condiciones de la vivienda y H.Fuerza de trabajo (uso del tiempo e ingresos) de la siguiente [página](http://microdatos.dane.gov.co/index.php/catalog/214/get_microdata).
En esta misma página, en la pestaña “Descripción de variables” podrá encontrar una descripción detallada de las variables que contiene las dos bases de datos.
2. Importe las dos bases de datos a Jupyter Notebook.
3. Para cada uno de las bases de datos realice:
    -  Realice un análisis del tipo de variables que contiene las bases de datos (si son cuantitativas o cualitativas), para esto, utilice la función `df.dtypes` (ver práctica 27 de agosto), si el tipo de un variable no es consistente, realizar la respectiva corrección.
    - Realice un análisis de datos duplicados y tome una decisión con relación a estos (eliminarlos o no), justifique su respuesta.
    - Determine el porcentaje de datos faltantes por cada una de las variables y tome una acción con relación a estos (eliminar las variables que tienen datos missing, eliminar los registros con datos missing, etc). Justifique su respuesta.
    - Para cada una de las bases de datos, seleccione un conjunto de variables que usted considere de interés, realice un análisis sobre los posibles valores que pueden tener esta variable, para esto utilice la función `df.variable.unique()`, donde df es el nombre del dataframe que contiene una base de datos (ver práctica 3 de septiembre). Si en una variable contiene valores que no son lógicos  ¿qué acción realizaría sobre esta base de datos? Justifique su respuesta.

4. De la misma página del punto 1, descargue un tercer conjunto de datos (cualquiera que sea interesante para usted) y replique los puntos 3.1, 3.2, y 3.3.
5. Con las tres bases de datos obtenidas de los puntos 3 y 4, realice un tipo de JOIN (left, right, inner, etc), y a partir de esta base consolidada, realice un análisis muy general de las variables utilizando la función pandas profiling.
6. ¿Qué tipo de restricciones legales pueden consideran pueden tener al momento de trabajar sobre estos datos?
7. Plantear tres hipótesis de relevancia para el negocio a la luz del negocio.

## **1. Descarga de las bases de datos**

In [1]:
!wget -q "https://github.com/jpcano1/MINE-4101-tareas/blob/master/datasets/CAPITULO%20B.%20CONDICIONES%20DE%20LA%20VIVIENDA.sav?raw=true" -O "condiciones_vivienda.sav"
!wget -q "https://github.com/jpcano1/MINE-4101-tareas/blob/master/datasets/CAPITULO%20H.%20FUERZA%20DE%20TRABAJO%20-%20Uso%20del%20tiempo%20e%20ingresos.sav?raw=true" -O "fuerza_trabajo.sav"
!wget -q "https://github.com/jpcano1/MINE-4101-tareas/blob/master/datasets/CAPITULO%20D.%20COMPOSICION%20DEL%20HOGAR.sav?raw=true" -O "composicion_hogar.sav"

In [ ]:
!pip install -q pyreadstat
!pip install -q https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

## **2. Importación de las bases de datos**

In [4]:
data_condiciones = pd.read_spss("condiciones_vivienda.sav")
data_fuerza = pd.read_spss("fuerza_trabajo.sav")

In [5]:
columns_condiciones = {
    "FEX_C": "factores_expansion",
    "P424": "tipo_vivienda",
    "P4030S1": "energia_electrica",
    "P4030S1A1": "estrato",
    "P4030S5": "acueducto",
    "P4030S3": "alcantarillado",
    "P4030S4": "recoleccion_basuras",
    "P4030S4A1": "dias_x_semana",
    "P4030S2": "gas_natural_dom",
    "P4020": "material_pisos",
    "DIA_REFERENCIA_2": "dia_referencia"
}

In [6]:
data_condiciones.rename(columns=columns_condiciones, inplace=True)
data_condiciones.rename(columns=lambda x: x.lower(), inplace=True)

In [7]:
columns_fuerza_list = ["oficio_princ", "rama_princ", 
                       "lugar_trabajo", "contrato", "misma_empresa", 
                       "horas_semana", "trabajo_dia", "horas_trabajo", 
                       "minutos_trabajo",  "total_ida", "horas_ida", 
                       "minutos_ida",  "motivo_no_trabajo", "este_trabajo", 
                       "ganancias_mes_pasado", "ganancia_neta_mes_pasado", 
                       "numero_meses", "cantidad_tiempo", "numero_personas",
                       "empleo_secundario", "horas_semana_empleo_secundario", 
                       "trabajo_secundario_dia", "horas_trabajo_secundario", 
                       "minutos_trabajo_secundario", "ida_secundario_horas", 
                       "ida_secundario_minutos",  "ganancia_secundaria_mes_pasado", 
                       "ganancia_jubilacion", "valor_jubilacion", "ganancia_arriendo",  
                       "valor_arriendo",  "ganancia_ayudas", "ayuda_provino_de", 
                       "valor_ayudas"]

In [8]:
index = 0
columns_fuerza = {}
for col in data_fuerza.columns[3:]:
    columns_fuerza[col] = columns_fuerza_list[index]
    index += 1

In [42]:
columns_fuerza

{'P1098': 'motivo_no_trabajo',
 'P1099': 'total_ida',
 'P1099S1': 'horas_ida',
 'P1099S2': 'minutos_ida',
 'P1145': 'ganancia_ayudas',
 'P1145S1': 'ayuda_provino_de',
 'P1145S2': 'valor_ayudas',
 'P1148S1': 'ida_secundario_horas',
 'P1148S2': 'ida_secundario_minutos',
 'P1149': 'trabajo_secundario_dia',
 'P1149S1': 'horas_trabajo_secundario',
 'P1149S2': 'minutos_trabajo_secundario',
 'P1150': 'trabajo_dia',
 'P1150S1': 'horas_trabajo',
 'P1150S2': 'minutos_trabajo',
 'P1151': 'horas_semana',
 'P427': 'este_trabajo',
 'P428': 'numero_personas',
 'P6250': 'empleo_secundario',
 'P6370S1': 'oficio_princ',
 'P6390S1': 'rama_princ',
 'P6400': 'misma_empresa',
 'P6426': 'cantidad_tiempo',
 'P6440': 'contrato',
 'P6500': 'ganancias_mes_pasado',
 'P6510': 'ganancia_jubilacion',
 'P6510S1': 'valor_jubilacion',
 'P6545': 'ganancia_arriendo',
 'P6545S1': 'valor_arriendo',
 'P6750': 'ganancia_neta_mes_pasado',
 'P6760': 'numero_meses',
 'P6850': 'horas_semana_empleo_secundario',
 'P6880': 'lugar_t

In [9]:
data_fuerza.rename(columns=columns_fuerza, inplace=True)
data_fuerza.rename(columns=lambda x: x.lower(), inplace=True)

### **3. Análisis de datos**
- Tercer Punto

## **3.1. Análisis del tipo de variable**
- Análisis para condiciones de vivienda:

In [10]:
data_condiciones.dtypes

directorio              float64
factores_expansion      float64
region                 category
dia_referencia         category
clase                  category
tipo_vivienda          category
energia_electrica      category
estrato                 float64
acueducto              category
alcantarillado         category
recoleccion_basuras    category
dias_x_semana           float64
gas_natural_dom        category
material_pisos         category
dtype: object

- Análisis para fuerza de trabajo

In [11]:
data_fuerza.dtypes

directorio                         float64
secuencia_p                        float64
orden                              float64
oficio_princ                        object
rama_princ                          object
lugar_trabajo                     category
contrato                          category
misma_empresa                     category
horas_semana                       float64
trabajo_dia                       category
horas_trabajo                      float64
minutos_trabajo                    float64
total_ida                          float64
horas_ida                          float64
minutos_ida                        float64
motivo_no_trabajo                 category
este_trabajo                      category
ganancias_mes_pasado               float64
ganancia_neta_mes_pasado           float64
numero_meses                       float64
cantidad_tiempo                    float64
numero_personas                   category
empleo_secundario                 category
horas_seman

### **3.2 Análisis de duplicados**
- Dado que la mayoría de las respuestas de la encuesta realizada son de opción multiple, lo más seguro es que esté llena de duplicados. Por lo que, lo mejor sería no borrarlos, porque cada respuesta, aunque duplicada, nos da una visión de la tendencia de respuestas.

### **3.3 Análisis de datos faltantes**

In [12]:
def null_percentage(df, int_mode=False):
    null_percentages = dict()
    for col in df.columns:
        percentage = df[col].isna().sum() * 100
        percentage /= df.shape[0]
        if not int_mode:
            percentage = f"{percentage}% of null values"
        null_percentages[col] = percentage
    return null_percentages

In [13]:
condiciones_null = null_percentage(data_condiciones)

- Dado que este dataset está muy limpio de variables nulas, se dejarán todas para ver qué conclusiones se pueden sacar.

In [14]:
condiciones_null

{'acueducto': '0.0% of null values',
 'alcantarillado': '0.0% of null values',
 'clase': '0.0% of null values',
 'dia_referencia': '0.0% of null values',
 'dias_x_semana': '17.18576327302826% of null values',
 'directorio': '0.0% of null values',
 'energia_electrica': '0.0% of null values',
 'estrato': '2.062196996570888% of null values',
 'factores_expansion': '0.0% of null values',
 'gas_natural_dom': '0.0% of null values',
 'material_pisos': '0.0% of null values',
 'recoleccion_basuras': '0.0% of null values',
 'region': '0.0% of null values',
 'tipo_vivienda': '0.0% of null values'}

- Dado que este set de datos tiene más variables nulas, se procederá a eliminar aquellas variables con un porcentaje de nulidad mayor al 85% para analizar las variables que tienen más datos. Para las filas más de 10 valores nulos las vamos a eliminar también.

In [15]:
fuerza_null = null_percentage(data_fuerza)

In [16]:
fuerza_null

{'ayuda_provino_de': '88.29348989587726% of null values',
 'cantidad_tiempo': '48.796587628531576% of null values',
 'contrato': '48.796587628531576% of null values',
 'directorio': '0.0% of null values',
 'empleo_secundario': '48.796587628531576% of null values',
 'este_trabajo': '48.796587628531576% of null values',
 'ganancia_arriendo': '0.0% of null values',
 'ganancia_ayudas': '0.0% of null values',
 'ganancia_jubilacion': '0.0% of null values',
 'ganancia_neta_mes_pasado': '78.08556878263973% of null values',
 'ganancia_secundaria_mes_pasado': '97.5493853190178% of null values',
 'ganancias_mes_pasado': '72.18041454474682% of null values',
 'horas_ida': '61.526484803269646% of null values',
 'horas_semana': '48.796587628531576% of null values',
 'horas_semana_empleo_secundario': '97.5493853190178% of null values',
 'horas_trabajo': '61.526484803269646% of null values',
 'horas_trabajo_secundario': '98.82172629666871% of null values',
 'ida_secundario_horas': '98.82172629666871% o

In [17]:
fuerza_null = null_percentage(data_fuerza, int_mode=True)

In [18]:
def drop_null_column(df, null_percentages, drop_perc=85):
    for col in df.columns:
        if null_percentages[col] > drop_perc:
            df.drop(columns=col, inplace=True)

In [19]:
drop_null_column(data_fuerza, fuerza_null)

In [20]:
fuerza_null = null_percentage(data_fuerza)

In [21]:
fuerza_null

{'cantidad_tiempo': '48.796587628531576% of null values',
 'contrato': '48.796587628531576% of null values',
 'directorio': '0.0% of null values',
 'empleo_secundario': '48.796587628531576% of null values',
 'este_trabajo': '48.796587628531576% of null values',
 'ganancia_arriendo': '0.0% of null values',
 'ganancia_ayudas': '0.0% of null values',
 'ganancia_jubilacion': '0.0% of null values',
 'ganancia_neta_mes_pasado': '78.08556878263973% of null values',
 'ganancias_mes_pasado': '72.18041454474682% of null values',
 'horas_ida': '61.526484803269646% of null values',
 'horas_semana': '48.796587628531576% of null values',
 'horas_trabajo': '61.526484803269646% of null values',
 'lugar_trabajo': '48.796587628531576% of null values',
 'minutos_ida': '61.526484803269646% of null values',
 'minutos_trabajo': '61.526484803269646% of null values',
 'misma_empresa': '72.68562068182555% of null values',
 'numero_meses': '78.08556878263973% of null values',
 'numero_personas': '48.79658762853

In [28]:
data_fuerza.dropna(thresh=15, inplace=True)

In [30]:
null_percentage(data_fuerza)

{'cantidad_tiempo': '0.0% of null values',
 'contrato': '0.0% of null values',
 'directorio': '0.0% of null values',
 'empleo_secundario': '0.0% of null values',
 'este_trabajo': '0.0% of null values',
 'ganancia_arriendo': '0.0% of null values',
 'ganancia_ayudas': '0.0% of null values',
 'ganancia_jubilacion': '0.0% of null values',
 'ganancia_neta_mes_pasado': '57.20122897595895% of null values',
 'ganancias_mes_pasado': '45.66849323746476% of null values',
 'horas_ida': '24.861423458236988% of null values',
 'horas_semana': '0.0% of null values',
 'horas_trabajo': '24.861423458236988% of null values',
 'lugar_trabajo': '0.0% of null values',
 'minutos_ida': '24.861423458236988% of null values',
 'minutos_trabajo': '24.861423458236988% of null values',
 'misma_empresa': '46.6551582148174% of null values',
 'numero_meses': '57.20122897595895% of null values',
 'numero_personas': '0.0% of null values',
 'oficio_princ': '0.0% of null values',
 'orden': '0.0% of null values',
 'rama_pri

### **3.4 Análisis de variables restantes**
- Para el dataset de condiciones voy a usar las variables "estrato", "acueducto", "region", "alcantarillado" y "gas_natural_dom" para analizar las regiones y los estratos que cuentan con los servicios ya nombrados.

In [29]:
data_condiciones.head()

,directorio,factores_expansion,region,dia_referencia,clase,tipo_vivienda,energia_electrica,estrato,acueducto,alcantarillado,recoleccion_basuras,dias_x_semana,gas_natural_dom,material_pisos
0,2915591.0,463.993402,Central,Domingo,Cabecera,Casa,Sí,3.0,Sí,Sí,Sí,3.0,Sí,"Cemento, gravilla"
1,2915592.0,93.084381,Pacífica,Miércoles,Cabecera,Casa,Sí,1.0,No,No,Sí,3.0,No,"Baldosa, cerámica, laminado"
2,2915593.0,516.279638,Central,Domingo,Cabecera,Casa,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Baldosa, cerámica, laminado"
3,2915594.0,655.671443,Central,Domingo,Cabecera,Apartamento,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Cemento, gravilla"
4,2915595.0,326.783507,Central,Lunes,Cabecera,Apartamento,Sí,1.0,Sí,Sí,Sí,3.0,Sí,"Baldosa, cerámica, laminado"


In [50]:
data_condiciones = data_condiciones.query("estrato!=9")

In [51]:
data_condiciones["estrato"].unique()

array([ 3.,  1.,  2.,  4.,  0., nan,  5.,  6.])

In [52]:
data_condiciones["region"].unique()

[Central, Pacífica, Oriental, Atlántica, San Andrés, Bogotá]
Categories (6, object): [Central, Pacífica, Oriental, Atlántica, San Andrés, Bogotá]

In [53]:
data_condiciones["acueducto"].unique()

[Sí, No]
Categories (2, object): [Sí, No]

In [54]:
data_condiciones["alcantarillado"].unique()

[Sí, No]
Categories (2, object): [Sí, No]

In [55]:
data_condiciones["gas_natural_dom"].unique()

[Sí, No]
Categories (2, object): [Sí, No]

- Del dataset de fuerza de trabajo voy a utilizar las columnas horas por semana, contrato y trabajo secundario

In [56]:
data_fuerza.head()

,directorio,secuencia_p,orden,oficio_princ,rama_princ,lugar_trabajo,contrato,misma_empresa,horas_semana,trabajo_dia,horas_trabajo,minutos_trabajo,horas_ida,minutos_ida,este_trabajo,ganancias_mes_pasado,ganancia_neta_mes_pasado,numero_meses,cantidad_tiempo,numero_personas,empleo_secundario,ganancia_jubilacion,ganancia_arriendo,ganancia_ayudas
0,2915591.0,1.0,1.0,53,5529,En esta vivienda,No.,NaN,4.0,Sí,4.0,0.0,0.0,0.0,Trabajador por cuenta propia,NaN,90000.0,1.0,12.0,Trabaja solo,No,No,No.,No.
1,2915592.0,1.0,1.0,62,113,"En el campo o área rural, mar o río",No.,NaN,12.0,No.,NaN,NaN,NaN,NaN,Trabajador de su propia finca o de finca en ar...,NaN,300000.0,1.0,96.0,2 a 4 personas,No,Sí,No.,No.
2,2915593.0,1.0,1.0,79,1810,En esta vivienda,No.,NaN,30.0,No.,NaN,NaN,NaN,NaN,Trabajador por cuenta propia,NaN,200000.0,1.0,84.0,Trabaja solo,No,Sí,No.,Sí
4,2915594.0,1.0,1.0,85,5272,"Local fijo, oficina, fábrica, etc.",No.,NaN,25.0,No.,NaN,NaN,NaN,NaN,Trabajador por cuenta propia,NaN,200000.0,1.0,300.0,2 a 4 personas,Sí,No,No.,No.
5,2915594.0,1.0,2.0,77,1551,En esta vivienda,No.,NaN,20.0,No.,NaN,NaN,NaN,NaN,Trabajador por cuenta propia,NaN,300000.0,1.0,72.0,2 a 4 personas,No,No,No.,Sí


In [57]:
data_fuerza["contrato"].unique()

[No., Sí]
Categories (2, object): [No., Sí]

In [58]:
data_fuerza["horas_semana"].unique()

array([  4.,  12.,  30.,  25.,  20.,  45.,  48.,  40.,  75.,  50.,  70.,
         8.,  54.,  36.,  44.,   6.,  21.,  84.,  14.,  10.,  28.,  56.,
         9.,  18.,  55.,  96.,  42.,  60.,  35., 100.,  72.,  98.,  80.,
        11.,  24.,  49.,  77.,  27.,  46.,  63.,  32.,  64.,  31., 105.,
        37.,   3.,  16.,  15.,  71.,  59., 126.,  58.,   5.,   7.,  33.,
         1., 112.,   2.,  66.,  38.,  52.,  68.,  85.,  78.,  22.,  76.,
        90.,  53.,  83.,  86., 140.,  43.,  67.,  39.,  65.,  17.,  62.,
        92.,  41., 119.,  23.,  95.,  79.,  91., 110., 108.,  51.,  73.,
        47.,  13.,  89., 120., 107.,  19.,  82.,  57.,  97.,  94.,  87.,
        61., 114., 133.,  81.,  74.,  29.,  26., 102.,  34., 104.,  69.,
        88., 115., 118., 116.,  93., 132., 117., 101., 136., 103., 127.,
       123., 106., 125., 109., 128., 113.,  99., 130.])

In [59]:
data_fuerza["empleo_secundario"].unique()

[No, Sí]
Categories (2, object): [No, Sí]

## **4. Adquisición del tercer dataset**

In [60]:
data_composicion = pd.read_spss("composicion_hogar.sav")

In [61]:
data_composicion.head()

,DIRECTORIO,SECUENCIA_P,ORDEN,P6040,P6020,P425,P1174,P1174S1,P1174S1A1,P1174S2,P1174S2A1,P1174S3,P1174S3A1,P1173,P426,P1172,P1172S1,P5762,P5762S2,P5754,P5754S2
0,2915591.0,1.0,1.0,51.0,Mujer,Jefe(a) del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ninguno de los anteriores,Está separado(a) o divorciado(a),NaN,NaN,No,NaN,No,NaN
1,2915592.0,1.0,1.0,63.0,Mujer,Jefe(a) del hogar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Negro, mulato (afrodescendiente)",Está separado(a) o divorciado(a),NaN,NaN,No,NaN,No,NaN
2,2915593.0,1.0,1.0,38.0,Mujer,Jefe(a) del hogar,No,NaN,NaN,NaN,NaN,NaN,NaN,Ninguno de los anteriores,Está separado(a) o divorciado(a),NaN,NaN,No,NaN,No,NaN
3,2915593.0,1.0,2.0,15.0,Mujer,"Hijo(a), Hijastro(a)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ninguno de los anteriores,Está soltero(a),NaN,NaN,Sí,1.0,No,NaN
4,2915593.0,1.0,3.0,7.0,Mujer,"Hijo(a), Hijastro(a)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ninguno de los anteriores,NaN,NaN,NaN,Sí,1.0,No,NaN


In [63]:
composicion_null = null_percentage(data_composicion, int_mode=True)

In [67]:
drop_null_column(data_composicion, composicion_null, drop_perc=85)

In [71]:
null_percentage(data_composicion)

{'DIRECTORIO': '0.0% of null values',
 'ORDEN': '0.0% of null values',
 'P1172': '61.63227648627535% of null values',
 'P1172S1': '63.685585755461574% of null values',
 'P1173': '0.0% of null values',
 'P1174': '74.51377852005496% of null values',
 'P425': '0.0% of null values',
 'P426': '20.26439134768203% of null values',
 'P5754': '0.0% of null values',
 'P5754S2': '73.66524795948604% of null values',
 'P5762': '0.0% of null values',
 'P5762S2': '56.36330576731407% of null values',
 'P6020': '0.0% of null values',
 'P6040': '0.0% of null values',
 'SECUENCIA_P': '0.0% of null values'}

In [75]:
data_composicion.dropna(thresh=10, inplace=True)